In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd

# df = pd.read_csv('drive/My Drive/tweets_monolingual_5p.csv')
df = pd.read_csv('../data/tweets_monolingual_5p.csv')
df.shape

(69170, 7)

In [31]:
df.head()

,Unnamed: 0,id_str,screen_name,raw_text,chn_text,eng_text,monolingual_text
0,0,1193152895337689088,YXSzzzz,@gww067 Haha there is no way Ryza is not cute 😂,NaN,Haha there is no way Ryza is not cute,Haha there is no way Ryza is not cute
1,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,乳不平何以平天下!
2,3,1193152895337689088,YXSzzzz,@HrJasn 乳不巨何以聚人心 (誤,乳不巨何以聚人心誤,NaN,乳不巨何以聚人心誤
3,4,1193152895337689088,YXSzzzz,因為蠻多人問。。。我就統一發個 😂\nBecuz too many people ask ....,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…,Becuz too many people ask ... I will post thi...,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…
4,5,1193152895337689088,YXSzzzz,因為蠻多人問。。。我就統一發個 😂\nBecuz too many people ask ....,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…,Becuz too many people ask ... I will post thi...,Becuz too many people ask ... I will post thi...


In [ ]:
df.drop('Unnamed: 0', inline=True)

In [32]:
df.screen_name.unique()

array(['YXSzzzz', 'GreenBright2', 'ZhuLiuHai1', 'SangPu15', 'BlessHK3',
       '0318May', '2nmnmcvkhKwByww', 'RainbowRainRai1', 'kacavawu',
       'Lucien86734903', 'yueming02341069', 'susanzh77455188',
       'Abby23079586', 'CherieHuang3', 'yuenck3', 'hkunderwearmen',
       'Iming9999', 'lee_tungtung', 'ZazousLes', 'Heerfei',
       'AnnaRao32559926', 'amyfox2009', 'pyequality', 'dgchuanghe',
       'EmmaHaoTheBest', 'kevin77757997', 'YingShi55', 'Wangcat11',
       'KP_Taipei', 'Nan_Is_Me', 'kelvinchan4335', 'EdrWx',
       'Shevonne1207', 'steffenroski', 'cla12a2013', 'bluerightking',
       'thenapoleonhung', 'easoncxz', 'siumou', 'yipcw',
       'SVWomanEngineer', 'kk23wong', 'calvynation', 'AndyCoach',
       'denistsao', 'Leifson', 'hongliji', 'sunbear', 'TrueOfLifeTP',
       'poiesis'], dtype=object)

In [13]:
len(df.screen_name.unique())

50

## Vectorize the tweets

In [5]:
!pip3 install polyglot PyICU pycld2 morfessor

### Prepare Monolingual Texts

In [14]:
df['monolingual_text'] = df[['chn_text', 'eng_text']].values.tolist()
df = df.explode('monolingual_text')
df = df.reset_index(drop=True)
df = df.dropna(subset=['monolingual_text'])
df = df.reset_index(drop=True)
df.head()

,Unnamed: 0,Unnamed: 0.1,id_str,screen_name,raw_text,chn_text,eng_text,translated_chn,monolingual_text
0,0,0,1193152895337689088,YXSzzzz,@gww067 Haha there is no way Ryza is not cute 😂,NaN,Haha there is no way Ryza is not cute,哈哈，不可能瑞紮不可愛,Haha there is no way Ryza is not cute
1,1,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,不太繁忙的測試。,乳不平何以平天下!
2,1,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,不太繁忙的測試。,Less busty testing.
3,2,2,1193152895337689088,YXSzzzz,@HrJasn 乳不巨何以聚人心 (誤,乳不巨何以聚人心誤,NaN,NaN,乳不巨何以聚人心誤
4,3,3,1193152895337689088,YXSzzzz,因為蠻多人問。。。我就統一發個 😂\nBecuz too many people ask ....,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…,Becuz too many people ask ... I will post thi...,因為太多人問...我會張貼這個Mod這是我的圖紙，而fe,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…


In [15]:
df.shape

(73860, 9)

### Prepare Translated Monolingual Texts

In [25]:
df['eng_text'].fillna(df['translated_eng'], inplace=True)
df.eng_text = df.eng_text.astype(str)
df.head()

,Unnamed: 0,id_str,screen_name,raw_text,chn_text,eng_text,translated_eng
0,0,1193152895337689088,YXSzzzz,@gww067 Haha there is no way Ryza is not cute 😂,NaN,Haha there is no way Ryza is not cute,NaN
1,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,Why is the milk uneven in the world!
2,2,1193152895337689088,YXSzzzz,@HrJasn 乳不巨何以聚人心 (誤,乳不巨何以聚人心誤,Why milk is not huge is the wrong thing to gat...,Why milk is not huge is the wrong thing to gat...
3,3,1193152895337689088,YXSzzzz,因為蠻多人問。。。我就統一發個 😂\nBecuz too many people ask ....,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…,Becuz too many people ask ... I will post thi...,Because quite a lot of people ask... I'll send...
4,4,1193152895337689088,YXSzzzz,PS: it was not my update. but good to see more...,NaN,PS: it was not my update. but good to see more...,NaN


### Prepare X, y, CV

In [26]:
from polyglot.text import Text
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold

X = df.eng_text
y = df.screen_name

def ploy_tokenizer(raw_text):
    return Text(raw_text).words

count_vectorizer = CountVectorizer(tokenizer=ploy_tokenizer, max_features=5000)
# tf_transformer = TfidfTransformer()

kf = KFold(n_splits=10, shuffle=True, random_state=1)

## Classification

In [27]:
import logging

from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics

logging.getLogger().setLevel(logging.ERROR)

nb = MultinomialNB()
lr = LogisticRegression(solver='liblinear', multi_class='auto', class_weight='balanced', verbose=1)
svm = LinearSVC(class_weight='balanced', verbose=1)

def tokenize_test_pipeline(model, dataset_X=X, dataset_y=y):
    pipe = Pipeline([
        ('count', count_vectorizer),
        # ('tfidf', tf_transformer),
        ('clf', globals()[model]),
    ])

    params = {
        'count__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'count__analyzer': ['word', 'char'],
    }

    algorithm = ''
    if model.startswith('lr'):
        algorithm = 'Logistic Regression'
    elif model.startswith('nb'):
        algorithm = 'Multinomial Naive Bayes'
    elif model.startswith('svm'):
        algorithm = 'Linear SVM'

    gd_clf = GridSearchCV(pipe, params, cv=10, scoring='accuracy', n_jobs=-1, verbose=1)
    gd_clf.fit(dataset_X, dataset_y)

    print(algorithm)
    print('------------------')
    print('Scores: ')
    print(gd_clf.cv_results_)
    print('Best params:')
    print(gd_clf.best_params_)
    print('Best score:')
    print(gd_clf.best_score_)

### Logistic Regression

In [11]:
tokenize_test_pipeline(model='lr')

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 62.7min finished
[LibLinear]Logistic Regression
------------------
Scores: 
{'mean_fit_time': array([132.14860823, 163.95448198, 210.12704864, 422.21671927,
       934.13132191, 916.00238762]), 'std_fit_time': array([  5.64906842,   9.36070949,  18.60320582,  21.6396404 ,
        25.21502694, 230.4503087 ]), 'mean_score_time': array([2.91949921, 5.36475735, 7.61303818, 0.51067312, 1.33316512,
       1.50437529]), 'std_score_time': array([0.10039169, 0.29739901, 0.28535073, 0.04520666, 0.21666141,
       0.47259682]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram

### SVM

In [13]:
tokenize_test_pipeline(model='svm')

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 32.6min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 61.0min finished
[LibLinear]Linear SVM
------------------
Scores: 
{'mean_fit_time': array([ 195.10204113,  206.81252863,  237.98372262, 1101.0949352 ,
        759.54852133,  314.18964348]), 'std_fit_time': array([ 6.44813057,  5.75966484,  3.55224823, 13.02302361,  8.46335838,
       67.01068921]), 'mean_score_time': array([3.10376823, 5.34769273, 8.16282222, 0.61721988, 1.49795084,
       1.78612366]), 'std_score_time': array([0.42781098, 0.74722465, 1.09875767, 0.02000622, 0.11999064,
       0.61484092]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': 

### Translated ENG Logistic Regression

In [28]:
tokenize_test_pipeline(model='lr')

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 98.4min finished
[LibLinear]Logistic Regression
------------------
Scores: 
{'mean_fit_time': array([ 108.69338543,  142.32223682,  176.59184439,  181.75002394,
       1579.79950054, 2035.36635232]), 'std_fit_time': array([  3.80517121,   2.89782448,   5.87818433,   9.42902107,
        54.29913221, 520.90896675]), 'mean_score_time': array([2.51365199, 4.07062821, 5.77338424, 0.49230125, 1.19133818,
       1.59018648]), 'std_score_time': array([0.26670071, 0.2429129 , 0.34958147, 0.0822156 , 0.06877389,
       0.59974833]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count_

### Translated ENG SVM

In [29]:
tokenize_test_pipeline(model='svm')

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 33.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 88.0min finished
[LibLinear]Linear SVM
------------------
Scores: 
{'mean_fit_time': array([ 199.86388612,  204.61199472,  208.59291446, 1194.05278988,
       1540.43381255,  744.51242266]), 'std_fit_time': array([ 4.43497799,  4.7003657 ,  2.30412426, 30.52803481, 86.92077743,
       99.0820755 ]), 'mean_score_time': array([2.19013524, 3.8466135 , 5.83341744, 0.46746995, 1.39227376,
       2.02988892]), 'std_score_time': array([0.29277496, 0.38413285, 0.81796802, 0.01848559, 0.19368091,
       0.78178129]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': 

### Translated CHN Logistic Regression

In [22]:
tokenize_test_pipeline(model='lr')

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 22.5min finished
[LibLinear]Logistic Regression
------------------
Scores: 
{'mean_fit_time': array([ 79.85924022, 106.08262389, 142.9701786 , 201.78887875,
       256.23497031, 229.90636399]), 'std_fit_time': array([ 1.91491963,  4.17896374, 13.28977827, 18.38440116,  7.02918066,
       59.42484926]), 'mean_score_time': array([2.37760949, 3.90294809, 6.08079286, 0.34631639, 0.70903943,
       0.72321827]), 'std_score_time': array([0.09669501, 0.14720386, 0.27737206, 0.03173112, 0.05290408,
       0.24711447]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range

### Translated CHN SVM

In [23]:
tokenize_test_pipeline(model='svm')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Fitting 10 folds for each of 6 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 19.6min finished
[LibLinear]Linear SVM
------------------
Scores: 
{'mean_fit_time': array([143.75854456, 154.623878  , 183.76461287, 157.76418879,
       131.1755291 , 123.27293158]), 'std_fit_time': array([ 4.44951464,  2.62774771,  3.81153094,  3.69266955,  1.68336143,
       22.46824012]), 'mean_score_time': array([2.87137532, 4.49891565, 6.832815  , 0.43380499, 0.8085367 ,
       0.80856166]), 'std_score_time': array([0.19062099, 0.53707166, 0.88287584, 0.02238096, 0.05979435,
       0.31407861]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': masked